# Using PH_EXE class

This notebook explains how to use the **PH_EXE** class inside *execution_ph* module. 

This class, given an ansatz and its parent hamiltonian, provided as a string  Pauli decomposition, computes the asociated ground state energy (that should be near zero).

In [ ]:
import logging

In [ ]:
logging.basicConfig(
    format='%(asctime)s-%(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    #level=logging.INFO
    level=logging.DEBUG
)


In [ ]:
logger = logging.getLogger('__name__')

In [ ]:
import os
import sys
sys.path.append("../")
import numpy as np

In [ ]:
# myQLM qpus
from qat.qpus import PyLinalg, CLinalg
qpu_c = CLinalg()
qpu_p = PyLinalg()
# QLM qpus
from qlmaas.qpus import LinAlg, MPS
qpu_qaass = LinAlg()
qpu_mps = MPS(lnnize =True)

Before doing the siumulation we need to perform several steps:

1. Ansatz Definition
2. Ansatz Solving
3. Parent Hamiltonian Computation

## 1. Ansatz Definition

First we need to create the myqlm cirucit implementation of the desired ansatz (see notebook **01_Ansatzes.ipynb**) 

In [ ]:
from ansatzes import ansatz_selector

In [ ]:
# Ansatz Configuration
ansatz = "simple01"
ansatz_conf = {
    'nqubits' : 6,
    'depth' : 2
}
filename = "ansatz_{}_dept_{}_nqubits_{}".format(ansatz, ansatz_conf["depth"], ansatz_conf["nqubits"])
# Create Ansatz Circuit
circuit = ansatz_selector(ansatz, **ansatz_conf)

In [ ]:
%qatdisplay circuit --svg

## 2. Ansatz Solution

Now we need to fix the parameters of the ansatz and solving it(see notebook **01_Ansatzes.ipynb**)

In [ ]:
from ansatzes import SolveCircuit

In [ ]:
folder = "Saving/"

In [ ]:
# Solving  Ansatz Configuration

# setting parameters of the ansatz
parameters = {v_ : 2 * np.pi * np.random.rand() for i_, v_ in enumerate(circuit.get_variables())}
solve_conf = {
    "qpu" : qpu_mps,
    "parameters" : parameters,
    "filename": folder + filename,
    "save": False        
}
# Solving Ansatz
solv_ansatz = SolveCircuit(circuit, **solve_conf)
solv_ansatz.run()

In [ ]:
solv_ansatz.state

## 3. Parent Hamiltonian Computation

With the obtained stat eof the ansatz we need to compute the Parent Hamiltonian and get its Pauli string decomposition.

In [ ]:
from parent_hamiltonian import PH

In [ ]:
# Create PH
# Get teh amplitudes
amplitudes = list(solv_ansatz.state["Amplitude"])
ph_conf = {
    "filename": folder + filename,
    "save": False          
}
ph_object = PH(amplitudes, **ph_conf)
ph_object.local_ph()

In [ ]:
ph_object.pauli_pdf

## 4. Computing PH ground state

Now we have all mandatory inputs so we can use **PH_EXE** class for computing the ground state energy of the parent hamiltonian of our input ansatz.
We need to provide for initializing the class:

* *ansatz*: myqlm circuit of the ansatz where the parameters fixed.
* *pauli_ph*: pandas DataFrame with the Pauli decomposition of the Parent Hamiltonian
* *nqubits*: number of qubits of the ansatz

Additionally othe parameters can be provides as keyword arguments (*kwargs*):
* *qpu*: myqlm QPU unit for simulating the ground state energ
* *nb_shots*: number of shots for measuring GSE.
* *truncation*: for truncating the number of Pauli strings. It is a positive number and all the Pauli coefficients where its absolute value is lower than $10^{-\text{truncation}}$ will be deleted

The **run** method should be used for the computation.

In [ ]:
from execution_ph import PH_EXE

In [ ]:
vqe_conf = {
    "qpu" : qpu_qaass,
    "nb_shots": 1000,
    "truncation": 6,
    "filename": folder + filename,
    "save": False        
}

ansatz_circuit = solv_ansatz.circuit
pauli_ph = ph_object.pauli_pdf
nqubits = ansatz_conf["nqubits"]
exe_ph = PH_EXE(ansatz_circuit, pauli_ph, nqubits, **vqe_conf)

In [ ]:
pauli_ph

In [ ]:
exe_ph.run()

The results will be stored in the attribute **pdf_result** (that will be a pandas DataFrame)

In [ ]:
exe_ph.pdf_result

In [ ]:
exe_ph.pdf_info